<a href="https://colab.research.google.com/github/Lamblador/IR_expert_system/blob/main/Koshelev_IR_expert_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IR spectral analysis of organic compounds via machine learning approach
By: Koshelev Daniil 

Based on some code form: https://github.com/rayguna/IR-Spectra-Analyzer

-Work in progress (last updated: 25/02/2023)-

A. MOTIVATION

IR spectroscopy is a versatile and a precise method for chemical identification because every compound exhibits unique IR spectral features; however, the spectral features tend to be ambiguous. A manual interpretation is often time-consuming and may lead to inconsistent, erroneous results. Hence, there is a need for a rapid, accurate, and reproducible alternative to a manual interpretation.

B. PURPOSE

The purpose of this program is to automate the IR spectrum interpretation routine of an known compound by assigning its absorption bands to the avalible chemical bonds or viration types in the molecule structure.

C. RATIONALE

The steps taken by this program are summarized in the form of a flowchart that is shown below:

1. The steps begin with the NIST chemical list.
2. With the aid of periodic table of elements, functional group class, and PeakUtils.py package, additional columns 
are generated (i.e., molecular weight Mw; elements, functional group class, and peaks list).
3. Using the additional compounds attributes, we can then perform filtering and EDA to reduce the number of entries 
on the list.
4. Next, we download the IR spectra of the chemicals shortlist by specifying the CAS number.
5. Using jcamp, scipy, and numpy packages, read and treat spectra.
6. Develop models and feed an unknown spectrum into the model to yield the guessed compounds.

NOTES


Schedule

In [ ]:
from IPython.display import display, Image, SVG, Math, YouTubeVideo
Image(filename='data/flowchart.png', width=800, height=600)